In [62]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
import numpy as np
from datetime import datetime, timedelta
import warnings

In [63]:
# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [64]:
df = pd.read_csv('../data_file/isolate_merge_df.csv')

In [65]:
test_df = pd.read_csv('../data_file/fill_na_test.csv')

### holiday 37부터 43은 이상한 양상을 띈다. 기존 holiday feature를 극솟값에 대해서 0이되게 수정

In [66]:
df['date_time']

0         2022-06-01 00:00:00
1         2022-06-01 01:00:00
2         2022-06-01 02:00:00
3         2022-06-01 03:00:00
4         2022-06-01 04:00:00
                 ...         
200971    2022-08-24 19:00:00
200972    2022-08-24 20:00:00
200973    2022-08-24 21:00:00
200974    2022-08-24 22:00:00
200975    2022-08-24 23:00:00
Name: date_time, Length: 200976, dtype: object

In [67]:
def find_holiday(df, i):
    building_df = df[df['building_number'] == i]
    building_df.loc[:, ['date_time']] = pd.to_datetime(building_df['date_time'], format='%Y-%m-%d %H:00:00')
    building_df.set_index('date_time', inplace=True)
    building_df_daily_max = building_df.resample('D').max()
    # 평균
    avg_pwr = (building_df_daily_max['power_consumption'].max() + building_df_daily_max['power_consumption'].min())/2
    
    # local minima 찾기
    comparator = np.less
    local_minima_indices = argrelextrema(building_df_daily_max['power_consumption'].values, comparator)
    local_minima = building_df_daily_max['power_consumption'].iloc[local_minima_indices]
    local_minima = local_minima[local_minima < avg_pwr]
    #print(local_minima)
    
    building_df.loc[:, 'holiday'] = 0
    for i in local_minima.index:
        building_df.loc[building_df.index[(building_df.index >= i) & (building_df.index  < (i + timedelta(days=1)))], 'holiday'] = 1
    
    return building_df
    
    

### test set은 다 0으로 수정(휴일없음)

In [68]:
for i in range(37, 43):
    test_df[test_df['building_number'] == i]['holiday'] = 0

/var/folders/__/r8r58jz17gj32tjwycl0m0hr0000gn/T/ipykernel_6364/1100254188.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[test_df['building_number'] == i]['holiday'] = 0


### train set 수정

In [69]:
for i in range(37, 43):
    building_df = find_holiday(df, i)
    df.loc[df['building_number'] == i, 'holiday'] = building_df['holiday'].values

In [70]:
test_df.to_csv('../data_file/fix_holiday_feature_test.csv', index=False)

In [71]:
df.to_csv('../data_file/fix_holiday_feature_train.csv', index=False)

In [72]:
test_df = pd.read_csv('../data_file/fix_holiday_feature_test.csv')

In [73]:
train_df = pd.read_csv('../data_file/fix_holiday_feature_train.csv')

In [7]:
train_df.columns

Index(['date_time', 'building_number', 'temperature', 'rainfall', 'windspeed',
       'humidity', 'power_consumption', 'building_type', 'total_area',
       'cooling_area', 'solar_power_capacity', 'day', 'week_of_month',
       'day_hour_mean', 'hour_mean', 'hour_std', 'rainfall_any',
       'discomfort_index', 'cdh', 'week', 'sin_time', 'cos_time', 'holiday',
       'upper_outliers', 'lower_outliers', 'outliers', 'anomaly'],
      dtype='object')